# Practical session demo

We are going to take a look at the different functionalities:
* Create a design of experiments
* Sample from this space
* Evaluate the samples using a well-known benchmark function
* Iteratively find the minimum of the loss-landscape using a global optimizer
* Look at one of the example experiments: 20D Ackley optimization with CMAES

import the `f3dasm` package, and `numpy`:

In [ ]:
import f3dasm
import numpy as np

make sure that we specify a `seed`:

In [ ]:
seed = 42

### Design: Creating the design space

**Parameters**

There are three different parameters you can construct:
* We can create continuous parameters with a `lower_bound` and `upper_bound` (optional) with the `f3dasm.ContinuousParameter` class
* We can create discrete parameters with a `lower_bound` and `upper_bound` with the `f3dasm.DiscreteParameter` class
* We can create categorical parameters with a list of strings (categories) with the `f3dasm.CategoricalParameter` class

all parameters also require a `name`.

**Design space**

Parameters define the `f3dasm.DesignSpace` and can be constructed by calling the `f3dasm.DesignSpace` class and providing:
* a list of input parameters (`input_space`) 
* a list of output parameters (`output_space`)


***
**Exercise #1**

Create a `DesignSpace` with the following features:
* 2 continuous input parameters named `x0` and `x1`, both with the same range of `-1.0` to `1.0`
* 1 continuous output parameter named `y`
* Call this designspace `design`
***

In [ ]:
## YOUR CODE HERE ##

In [ ]:
## SOLUTION ##

# Input parameters
x1 = f3dasm.ContinuousParameter(name="x0", lower_bound=-1., upper_bound=1.)
x2 = f3dasm.ContinuousParameter(name="x1", lower_bound=-1., upper_bound=1.)

# Output parameters
y = f3dasm.ContinuousParameter(name="y")

# Create lists of parameters
input_space = [x1, x2]
output_space = [y]

# Construct the design space
design = f3dasm.DesignSpace(input_space=input_space, output_space=output_space)

In [ ]:
## CHECK ##

assert design == f3dasm.make_nd_continuous_design(bounds=np.tile([-1., 1.], (2,1)), dimensionality=2)

What's next? We can sample from this design space with the **sampling** block!

### Sampling: Latin Hypercube Sampler

Samplers can be found within the submodule `f3dasm.sampling`. Currently 3 samplers are readily available:
* Random Uniform Sampling (`f3dasm.sampling.RandomUniformSampling`)
* Latin Hypercube Sampling(`f3dasm.sampling.LatinHypercubeSampling`)
* Sobol Sequence Sampling (`f3dasm.sampling.SobolSequenceSampling`)

**Creating a sampler object**
A new sampler can be created by initializing the sampler with:
* A design space
* A random seed (optional)

After that, you can sample by invoking the `get_samples(numsamples)`, where `numsamples` is the number of samples you want to get

***
<b>Exercise #2</b>

Now you are going to create a sampler and sample some points from the designspace:
* Create a <u>Random Uniform sampling</u> with the designspace and seed described above
* Sample 30 points from the designspace
* Store the resulting `Data`-object in the variable `samples_random`
* Do the same with <u>Latin Hypercube sampling</u> and store the samples as `samples_lhs`
***

In [ ]:
## YOUR CODE HERE ##



# If you are stuck you may uncomment the following lines to import the data object directly:
# samples_lhs = f3dasm.read_pickle('exercise_samples_lhs')
# samples_random = f3dasm.read_pickle('exercise_samples_random')

In [ ]:
## SOLUTION ##

# Construct sampler
random_sampler = f3dasm.sampling.RandomUniformSampling(design=design,seed=seed)
lhs_sampler = f3dasm.sampling.LatinHypercubeSampling(design=design, seed=seed)

N = 30 # Number of samples

# Get samples
samples_random = random_sampler.get_samples(numsamples=N)
samples_lhs = lhs_sampler.get_samples(numsamples=N)

In [ ]:
samples_random.data

In [ ]:
## CHECK ##

assert samples_lhs.data.equals(f3dasm.read_pickle('exercise_samples_lhs').data)
assert samples_random.data.equals(f3dasm.read_pickle('exercise_samples_random').data)

We can plot the datapoints with the `data.plot()` function:

In [ ]:
fig, ax = samples_lhs.plot(input_par1='x0', input_par2='x1')

In [ ]:
samples_random.data

As you could see earlier, the output values are all `NaN`. Let's evaluate the samples with a benchmark function from the **simulation** block!

### Simulation: Evaluating a benchmark function

Several benchmark functions have been implemented in the submodule `f3dasm.functions` to work with **continuous and single-objective** optimization problems.

These functions require an input-vector $\mathbf{x}$ and output a scalar $f(\mathbf{x})$

The `Levy()` function is a well-known, multimodal function:

$f(\textbf{x}) = \sin^2 (\pi w_1) + \sum_{i = 1}^{D - 1} (w_i - 1)^2 \left( 1 + 10 \sin^2 (\pi w_i + 1) \right) + (w_d - 1)^2 (1 + \sin^2 (2 \pi w_d)) \\ w_i = 1 + \frac{x_i - 1}{4}`
$


First we create such a function `f` by creating an object from the `f3dasm.functions.Levy` class

> *The original input domain is (-10, 10), but we scale these boundaries to the boundaries our input space*

In [ ]:
dim = 2
domain = np.tile([-1., 1.], (dim,1))

In [ ]:
f = f3dasm.functions.Levy(dimensionality=dim, seed=seed, scale_bounds=domain)

The global minima are known for these functions and can be accesed by the `get_global_minimum()` function:

In [ ]:
x_min, y_min = f.get_global_minimum(dim)
print(f"The global minimum is {y_min} at position {x_min}")

We can plot a three-dimensional loss-landscape of the two input parameters $x_0$ and $x_1$ with the `plot()` function:
* `px` denotes the resolution on each axis
* `domain` denotes the domain of the two axis to be plotted

In [ ]:
fig, ax = f.plot(px=100, domain=domain)

Evaluating the function is easy, just pass either:
* the `Data()` object to it: `get_input_data()`
* or a 2D numpy array: `get_input_data().to_numpy()`

.. and all the samples will be evaluated.
The output will be a `np.ndarray`

In [ ]:
x = samples_lhs.get_input_data()

By calling the `add_output` option, we can add individual columns or overwrite data to our DataFrame:

In [ ]:
samples_lhs.add_output(output=f(x))

In [ ]:
samples_lhs.show()

Let's store these samples under a new name `data`:

We can create a contour plot with the samples coordinates with the `plot_data()` function: 

In [ ]:
fig, ax = f.plot_data(samples_lhs,px=300,domain=domain)

> The red star will indicate the best sample



***
**Exercise #3**

* By consulting the documentation, find the location of the best sample obtainted by Latin Hypercube sampling (the red star)
* Store this value as `x_best`
***

In [ ]:
## YOUR CODE HERE ##


In [ ]:
## SOLUTION ##
x_best = samples_lhs.get_n_best_input_parameters_numpy(nosamples=1)

In [ ]:
## CHECK ##

assert (x_best == f3dasm.read_pickle('exercise_best_sample')).all()

Now we will take a look how we can find the minimum of this loss-function with an optimizer!

### Optimization: CMAES optimizer

Optimizers can be found in the submodule `f3dasm.optimization` and are ported from several libraries such as `GPyOpt`, `scipy-optimize`, `tensorflow` and `pygmo`

We will use the CMAES optimizer to find the minimum. We can find an implementation in the `f3dasm.optimization` module:

In [ ]:
optimizer = f3dasm.optimization.CMAES(data=samples_lhs)

By calling the `iterate()` method and specifying the function and the number of iterations, we will start the optimization process:

In [ ]:
optimizer.iterate(iterations=100, function=f)

After that, we can extract the data:

In [ ]:
cmaes_data = optimizer.extract_data()

... and create a contour plot again:

In [ ]:
fig, ax = f.plot_data(data=cmaes_data,px=300, domain=domain)

**Hyperparameters**

You can overwrite the default hyperparameters of each optimizer by supplying a dictionary to the optimizer initializer.
The parameters of the optimizer can be viewed by calling the `parameter` attribute:

In [ ]:
optimizer.parameter

***
**Exercise #4**
* Optimize the same `Levy()`-function for 100 iterations but now with the `Adam` optimizer
* Change the `learning_rate` parameter to `2e-3`
* Plot the loss-landscape and data with the `plot_data` function
* Extract the data and call it `adam_data`
***


In [ ]:
## YOUR CODE HERE ##


In [ ]:
## SOLUTIONS ##
optimizer_adam = f3dasm.optimization.Adam(data=samples_lhs, hyperparameters={'learning_rate': 2e-3}, seed=seed)
optimizer_adam.iterate(100, function=f)
adam_data = optimizer_adam.extract_data()

#f3dasm.write_pickle('exercise_adam_data', adam_data)

fig2, ax2 = f.plot_data(data=adam_data,px=300, domain=domain)

optimizer_adam.parameter

In [ ]:
## CHECK ##
import pandas as pd
assert pd.testing.assert_frame_equal(adam_data.data, f3dasm.read_pickle('exercise_adam_data').data)
# assert adam_data.data.equals(f3dasm.read_pickle('exercise_adam_data').data)

In [ ]:
d = f3dasm.read_pickle('exercise_adam_data')
adam_data.data

In [ ]:
d.data

### Experiment: Multiple realizations of SGA on 20D Ackley function

Now we take a look at an example of an experiment where use the following blocks to optimize a 20D Ackley function with the CMAES optimizer over 10 different realizations:

![title](blocks.png)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Define the blocks:
dimensionality = 20
iterations = 600
realizations = 10

hyperparameters= {} # If none are selected, the default ones are used

domain = np.tile([-1., 1.], (dimensionality,1))
design = f3dasm.make_nd_continuous_design(bounds=domain, dimensionality=dimensionality)
data = f3dasm.Data(design)

In [ ]:
# We can put them in a dictionary if we want

implementation = {
'realizations': realizations,
'optimizer': f3dasm.optimization.CMAES(data=data, hyperparameters=hyperparameters), 
'function': f3dasm.functions.Ackley(dimensionality=dimensionality, noise=False, scale_bounds=domain),
'sampler': f3dasm.sampling.LatinHypercubeSampling(design, seed=seed),
'iterations': iterations,
}

The `run_multiple_realizations()` function will be the pipeline of this experiment:

In [ ]:
results = f3dasm.run_multiple_realizations(**implementation)

In [ ]:
results.data[3]

In [ ]:
from cycler import cycler
cc = (cycler(color=list('bgrcmyk')) *
      cycler(linestyle=['-', '--', 'dotted']))

In [ ]:
def calc_mean_std(results):
    mean_y = pd.concat([d.get_output_data().cummin() for d in results], axis=1).mean(axis=1)
    std_y = pd.concat([d.get_output_data().cummin() for d in results], axis=1).std(axis=1)
    return mean_y, std_y

In [ ]:
def create_axis(results: f3dasm.OptimizationResult, ax):
    mean_y, std_y = calc_mean_std(results.data)

    ax.plot(mean_y, label=f"optimizer={results.optimizer}")

    #ax.fill_between(np.arange(len(mean_y)), mean_y-1.96*std_y, mean_y+1.96*std_y, alpha=.1)

    return ax

In [ ]:
def plot_results(results: f3dasm.OptimizationResult):
    fig, ax = plt.figure(figsize=(15,8)), plt.axes()

    ax.set_xlabel('iterations')
    ax.set_ylabel('f(x)')
    ax.set_yscale('log')
    ax.set_prop_cycle(cc)

    for index, res in enumerate([results]):
        ax = create_axis(res, ax)

    leg = ax.legend(loc='upper right', ncol=3, fancybox=True, shadow=True) #bbox_to_anchor=(0.5, 1.05)
    return fig, ax

In [ ]:
plot_results(results)

We can change a hyper-parameter such as the `population` and rerun the experiment with ease:

In [ ]:
sga_2 = f3dasm.optimization.CMAES(data=data, hyperparameters={'population': 15})

implementation.update()
results_2 = f3dasm.run_multiple_realizations(**implementation)

plot_results(results_2)

In [ ]:
implementation2 = {
'realizations': realizations,
'optimizer': f3dasm.optimization.PSO(data=data), 
'function': f3dasm.functions.Levy(dimensionality=dimensionality, noise=False, scale_bounds=domain),
'sampler': f3dasm.sampling.SobolSequenceSampling(design, seed=seed),
'iterations': iterations,
}

results_3 = f3dasm.run_multiple_realizations(**implementation2)

In [ ]:
plot_results(results_3)